This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.

In [16]:
import platform
import os
import pandas
import glob

In [12]:
from parsing_functions import *

In [20]:
# Different locations if running either on Linux or Windows
if 'Linux' in platform.system():
    BasePath = os.path.join(os.path.sep, 'home', 'habi', '2214')
elif 'Windows' in platform.system():
    BasePath = os.path.join('N:', os.sep)
Root = os.path.join(BasePath, os.sep, 'asdf')
Root = 'logfiles'
print('We are loading all the data from the folder %s' % Root)

We are loading all the data from the folder logfiles


In [21]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [22]:
# Get *all* log files
# Using os.walk is way faster than using recursive glob.glob, see DataWrangling.ipynb for details
# Not sorting the found logfiles is also making it quicker
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [23]:
Data

,LogFile
0,logfiles/cichlids/103637/proj/103637~01.log
1,logfiles/cichlids/103637/proj/103637.log
2,logfiles/cichlids/103637/proj/103637~02.log
3,logfiles/cichlids/103637/proj/103637~00.log
4,logfiles/cichlids/103637/rec/103637_rec.log
...,...
1264,logfiles/tooth-battalion/81/proj/Tooth081~01.log
1265,logfiles/tooth-battalion/81/proj/Tooth081~04.log
1266,logfiles/tooth-battalion/81/proj/Tooth081~00.log
1267,logfiles/tooth-battalion/81/proj/Tooth081.log


In [25]:
fulllog(Data.LogFile[0])

[System]
Scanner=SkyScan1272
Instrument S/N=15G09089-B
Software Version=1.4
Home Directory=C:\SkyScan1272\SkyScan1272
Source Type=HAMAMATSU_L11871_20
Camera Type=XIMEA xiRAY16
Camera Pixel Size (um)=7.4
Camera X/Y Ratio=1.0023
[User]
User Name=haberthu
Computer Name=ANAMIC02
[Acquisition]
Data Directory=D:\Results\EAWAG\103637\proj
Filename Prefix=103637~01
Filename Index Length=8
Number Of Files= 1921
Number Of Rows= 3280
Number Of Columns= 4904
Partial Width=OFF
Image crop origin X=0
Image crop origin Y=0
Camera binning=1x1
Image Rotation=-0.08200
Optical Axis (line)= 1740
Camera to Source (mm)=174.07267
Object to Source (mm)=82.33100
Source Voltage (kV)=  60
Source Current (uA)= 166
Image Pixel Size (um)=3.499972
Scaled Image Pixel Size (um)=3.499972
Image Format=TIFF
Depth (bits)=16
Reference Intensity=57000
Camera position=Close
Exposure (ms)=1981
Rotation Step (deg)=0.100
Use 360 Rotation=NO
Scanning position=15.049 mm
Frame Averaging=ON (3)
Random Movement=OFF (30)
Flat Field Co

()

We can also select a folder and go through *each* subfolder there...

In [22]:
# Different locations if running either on Linux or Windows
if 'Linux' in platform.system():
    BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-uct', 'Archiv_Tape')
#     BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
else:
    BasePath = os.path.join('R:\\')
# Jean
Root = os.path.join(BasePath, 'Rabbit-Grenoble')
# ZMK
Root = os.path.join(BasePath, 'ZahnmedizinischeKlinik', 'ToothBattallion')
Root = os.path.join(BasePath, 'ZMK', 'ToothBattallion')
# Christian
Root = os.path.join(BasePath, 'Lung*Muehlfeld')
# Tim/Ludovic
Root = os.path.join(BasePath, '*Melly*')
print('We are loading all the data from %s' % Root)

We are loading all the data from /home/habi/research-storage-uct/Archiv_Tape/*Melly*


In [102]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [103]:
# Look only for folders: https://stackoverflow.com/a/38216530
# Jean
# Data['Folder'] = sorted(glob.glob(os.path.join(Root, '**', 'Ra*' + os.path.sep)))
# ZMK
# Data['Folder'] = sorted(glob.glob(os.path.join(Root, '*' + os.path.sep)))
# Christian
# Data['Folder'] = sorted(glob.glob(os.path.join(Root, '19*' + os.path.sep)))
# Tim/Ludovic
Data['Folder'] = sorted(glob.glob(os.path.join(Root, 'Rat[67]*' + os.path.sep)))
Data['Scan'] = [os.path.basename(os.path.split(f)[0]) for f in Data['Folder']]

In [104]:
Data.head()

,Folder,Scan
0,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat60
1,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat61
2,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat62
3,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat63
4,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat64


In [105]:
# glob.glob(os.path.join(BasePath
#                        , '*' + os.path.sep))

In [106]:
# Grab a bunch of log files from a folder
# We could do it in a list comprehension, but then it fails if we're still scanning a tooth
# Data['LogFile'] = [sorted(glob.glob(os.path.join(f, '*.log')))[0] for f in Data['Folder']]
for c, row in Data.iterrows():
    try:
        # Jean & Christian
        # Data.at[c,'LogFile'] = sorted(glob.glob(os.path.join(row['Folder'], 'proj', '*.log')))[0]
        # ZMK
        # Data.at[c,'LogFile'] = sorted(glob.glob(os.path.join(row['Folder'], '*.log')))[0]
        # Tim/Ludovic
        Data.at[c,'LogFile'] = sorted(glob.glob(os.path.join(row['Folder'], 'overview', 'proj', '*.log')))[0]        
    except IndexError:
        print('No logfile found in %s, removing the folder' % row.Folder)
        Data.at[c,'LogFile'] = 'scanning'
Data = Data[Data['LogFile'] != 'scanning']
Data.reset_index(drop=True, inplace=True)
print('We have %s folders to work with' % (len(Data)))

We have 12 folders to work with


In [107]:
# # Choose the file selected manually
# log = raw.files[0]

In [108]:
# # Choose a file from the Dataframe with *all* log files
# log = Data.LogFile[3]

In [109]:
# print('We are looking at the values in %s' % log)

In [110]:
# Load log file manually (probably needed on Windows)
# log = 'f:\Verdiana Lung\Experiment01\Mouse04\proj\Mouse04.log'

In [111]:
# log = '/home/habi/research-storage-uct/Archiv_Tape/Rabbit-Grenoble/Rabbit-2-Grenoble2015/Rabbit-2-Overview-Grenoble2015-26-5um/proj/Rabbit-2-Overview-Grenoble2015-26-5um_.log'

In [47]:
# log = '/home/habi/1272/ZMK/ToothBattallion/99/Tooth099.log'

In [48]:
def fulllog(logfile):
    with open(logfile, 'r') as f:
        for line in f:
            print(line.strip())
    return()

In [49]:
# fulllog(log)

In [50]:
import re
def scanner(logfile, verbose=False):
    hardwareversion = []
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scanner' in line:
                if verbose:
                    print(line)
                # Sometimes it's SkyScan, sometimes Skyscan, so we have to regex it :)
                machine = re.split('Sky.can', line)[1].strip()
            if 'Hardware' in line:
                if verbose:
                    print(line)
                hardwareversion = line.split('=')[1].strip()
    if hardwareversion:
        return('SkyScan %s (Version %s)' % (machine, hardwareversion))
    else:
        return('SkyScan ' + machine)    

In [51]:
# scanner(log, verbose=True)

In [52]:
def controlsoftware(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Software' in line:
                if verbose:
                    print(line)
                version = line.split('=')[1].strip()
    return(version)

In [53]:
# fulllog(log)

In [57]:
# controlsoftware(log, verbose=True)

In [336]:
def source(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source T' in line:
                if verbose:
                    print(line)
                source = line.split('=')[1].strip()
                if 'HAMAMA' in source:
                    # We split the string at '_L' to separate HAMAMATSU_L118...
                    # Afterwards we properly capitalize HAMAMATSU and
                    # join the strings back with ' L' to get the beginning of the reference back
                    source = ' L'.join([s.capitalize() for s in source.split('_L')])
    return(source)

In [339]:
# source(log, verbose=True)

In [340]:
def camera(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Camera T' in line or 'Camera=' in line:
                if verbose:
                    print(line)
                cam = line.split('=')[1].strip().strip(' camera')
    return(cam)

In [341]:
# camera(log, verbose=True)

In [342]:
def voltage(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Voltage' in line:
                if verbose:
                    print(line)
                V = float(line.split('=')[1])
    return(V)

In [343]:
def current(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source Current' in line:
                if verbose:
                    print(line)
                A = float(line.split('=')[1])
    return(A)

In [344]:
# current(log, verbose=True)

In [345]:
def whichfilter(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                if verbose:
                    print(line)
                fltr = line.split('=')[1].strip().replace('  ', ' ')
                if fltr=='No Filter':
                    fltr=False
    return(fltr)

In [346]:
# whichfilter(log, verbose=True)

In [347]:
def numproj(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'f Files' in line:
                if verbose:
                    print(line)
                numproj = int(line.split('=')[1])
    return(numproj)

In [348]:
# numproj(log, verbose=True)

In [349]:
def stacks(logfile, verbose=False):
    with open(logfile, 'r') as f:
        # If only one stack, then there's nothing in the log file
        numstacks = 0
        for line in f:
            if 'b-scan' in line:
                if verbose:
                    print(line)
                # The 'Sub-scan scan length' is listed in the log file
                # We simply select the last one, and add 1, since Bruker also starts to count at zero
                numstacks = int(line.split('[')[1].split(']')[0])
    return(numstacks + 1)

In [350]:
# stacks(log, verbose=True)

In [351]:
def camerasize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Columns' in line:
                if verbose:
                    print(line)
                columns = int(line.split('=')[1])
            if 'Rows' in line:
                if verbose:
                    print(line)
                rows = int(line.split('=')[1])
    return(columns, rows)

In [352]:
def overlapscan(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Horizontal Off' in line:
                if verbose:
                    print(line)
                wide = int(line.split('=')[1])
                if wide == 1:
                    wide=False
    return(wide)

In [353]:
# TODO: Detect 360° scans!
# 'Use 360 Rotation=NO'

In [354]:
def rotationstep(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Rotation Step' in line:
                if verbose:
                    print(line)
                rotstep = float(line.split('=')[1])
    return(rotstep)

In [355]:
def exposure(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                if verbose:
                    print(line)
                exp = int(line.split('=')[1])
    return(exp)

In [356]:
def averaging(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Avera' in line:
                if verbose:
                    print(line)
                details = line.split('=')[1]
                if 'ON' in details:
                    # https://stackoverflow.com/a/4894156/323100
                    avg = int(details[details.find("(")+1:details.find(")")])
                else:
                    avg=False
    return(avg)

In [357]:
import datetime
def duration(logfile, verbose=False):
    '''Returns scantime in *seconds*'''
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scan duration' in line:
                if verbose:
                    print(line)
                duration = line.split('=')[1].strip()
    # Sometimes it's '00:24:26', sometimes '0h:52m:53s' :-/
    if 'h' in duration:
        scantime = datetime.datetime.strptime(duration, '%Hh:%Mm:%Ss')
    else:
        scantime = datetime.datetime.strptime(duration, '%H:%M:%S')
    return((scantime-datetime.datetime(1900,1,1)).total_seconds())

In [358]:
# log = '/home/habi/1272/Chondrules Space Yogita/NWA-200813/rec/NWA_rec.log'

In [359]:
def timeformat(tdelta, fmt):
    # From https://stackoverflow.com/a/8907269/323100
    d = {"days": tdelta.days}
    d["hours"], rem = divmod(tdelta.seconds, 3600)
    d["minutes"], d["seconds"] = divmod(rem, 60)
    return fmt.format(**d)

In [360]:
# log = Data.LogFile[66]

In [361]:
print('Each stack took', end=' ')
if duration(log)/3600 > 1:
    # Scan took hours
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
else:
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{minutes} minutes'), end=' ')
print('to scan')

Each stack took 2 hours and 48 minutes to scan


In [362]:
def pixelsize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                if verbose:
                    print(line)
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [363]:
def version(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Reconstruction Program' in line:
                if verbose:
                    print(line)
                Program = line.split('=')[1].strip()
            if 'Program Version' in line:
                if verbose:
                    print(line)
                Version = line.split('sion:')[1].strip()
    return(Program, Version)

In [364]:
def ringremoval(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring' in line:
                if verbose:
                    print(line)
                ring = int(line.split('=')[1].strip())
    return(ring)

In [365]:
# ringremoval(log, verbose=True)

In [366]:
def beamhardening(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'ardeni' in line:
                if verbose:
                    print(line)
                bh = int(line.split('=')[1].strip())
    return(bh)

In [367]:
# beamhardening(log, verbose=True)

In [368]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Software'] = [controlsoftware(log) for log in Data['LogFile']]

In [369]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Voxelsize_rounded'] = [round(vs,1) for vs in Data['Voxelsize']]

In [370]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]

In [371]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [372]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumProj'] = [numproj(log) for log in Data['LogFile']]
Data['CamSize'] = [camerasize(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Wide'] = [overlapscan(log) for log in Data.LogFile]

In [373]:
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]

In [374]:
Data['Exposure'] = [exposure(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [375]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]

In [376]:
Data['Version'] = [version(log) for log in Data['LogFile']]

In [377]:
Data.to_excel('Christian.xls')

In [378]:
Data.head()

,Folder,Scan,LogFile,Scanner,Software,Voxelsize,Voxelsize_rounded,Source,Camera,Voltage,...,NumProj,CamSize,RotationStep,Wide,RingRemoval,Beamhardening,Exposure,Averaging,Duration,Version
0,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat60,/home/habi/research-storage-uct/Archiv_Tape/He...,SkyScan 1272,1.1.19,10.000036,10.0,Hamamatsu L11871_20,XIMEA xiRAY16,80.0,...,948,"(2452, 1640)",0.2,False,14,0,2349,3,10100.0,"(NRecon, 1.7.4.6)"
1,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat61,/home/habi/research-storage-uct/Archiv_Tape/He...,SkyScan 1272,1.1.19,10.000036,10.0,Hamamatsu L11871_20,XIMEA xiRAY16,80.0,...,948,"(2452, 1640)",0.2,False,14,0,2349,3,10100.0,"(NRecon, 1.7.4.6)"
2,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat62,/home/habi/research-storage-uct/Archiv_Tape/He...,SkyScan 1272,1.1.19,10.000036,10.0,Hamamatsu L11871_20,XIMEA xiRAY16,80.0,...,948,"(2452, 1640)",0.2,False,14,0,2349,3,10180.0,"(NRecon, 1.7.4.6)"
3,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat63,/home/habi/research-storage-uct/Archiv_Tape/He...,SkyScan 1272,1.1.19,10.000036,10.0,Hamamatsu L11871_20,XIMEA xiRAY16,80.0,...,948,"(2452, 1640)",0.2,False,14,0,2349,3,10133.0,"(NRecon, 1.7.4.6)"
4,/home/habi/research-storage-uct/Archiv_Tape/He...,Rat64,/home/habi/research-storage-uct/Archiv_Tape/He...,SkyScan 1272,1.1.19,10.000036,10.0,Hamamatsu L11871_20,XIMEA xiRAY16,80.0,...,948,"(2452, 1640)",0.2,False,14,0,2349,3,10110.0,"(NRecon, 1.7.4.6)"


In [379]:
# for i in Data:
#     print(10 * '-', i, 20 * '-')
#     print(Data[i].unique())

In [380]:
# Christian is only interested in the fist two scans
# Data.drop(Data.index[2:], inplace=True)

----

My microct blurb from http://simp.ly/publish/NBhZhH

In [383]:
print('Based on the %s log files in %s' % (len(Data), Root))

Based on the 12 log files in /home/habi/research-storage-uct/Archiv_Tape/*Melly*


In [385]:
" OR ".join(str(value) for value in Data.Scanner.unique())

'SkyScan 1272'

In [386]:
print('After $PREPARATION, the',
      len(Data),
      'samples were imaged on a Bruker',
      " OR ".join(str(value) for value in Data.Scanner.unique()),
      'high-resolution microtomography machine (Control software version',
      " OR ".join(str(value) for value in Data.Software.unique()) + 
      ', Bruker microCT, Kontich, Belgium).')

After $PREPARATION, the 12 samples were imaged on a Bruker SkyScan 1272 high-resolution microtomography machine (Control software version 1.1.19, Bruker microCT, Kontich, Belgium).


In [388]:
print('The machine is equipped with a',
      " OR ".join(str(value) for value in Data.Source.unique()),
      'X-ray source and a',
      " OR ".join(str(value) for value in Data.Camera.unique()),
      'camera.')

The machine is equipped with a Hamamatsu L11871_20 X-ray source and a XIMEA xiRAY16 camera.


In [389]:
# if len(Data.Scanner.unique()) > 1:
#     print('more')

In [391]:
print('The X-ray source was set to a tube voltage of', 
      " OR ".join(str(value) for value in Data.Voltage.unique()),
      'kV and a tube current of',
      " OR ".join(str(value) for value in Data.Current.unique()),
      'µA, the x-ray spectrum was', end=' ')
if Data.Filter.unique():
    print('filtered by', " OR ".join(str(value) for value in Data.Filter.unique()), end=' ')
else:
    print('not filtered', end=' ')
print('prior to incidence onto the sample.')

The X-ray source was set to a tube voltage of 80.0 kV and a tube current of 125.0 µA, the x-ray spectrum was filtered by Al 1mm prior to incidence onto the sample.


In [392]:
# TODO: Flip the text of the filter to make it nicer

In [393]:
Data.Wide.unique()

array([False])

In [394]:
print('For each sample, we recorded a set of', end=' ')
if Data.Filter.unique():   
    print(" or ".join(str(value) for value in Data.Stacks.unique()),
          'stacked scans overlapping the sample height, each stack was recorded with', end=' ')
print(" or ".join(str(value) for value in Data.NumProj.unique()), 'projections of', end=' ')
for cs in Data.CamSize.unique():
    print(cs[0], end=' ')
print('x', end=' ')
for cs in Data.CamSize.unique():
    print(cs[1], end=' ')
print('pixels', end=' ')
if Data.Wide.unique():
    print('(' + " or ".join(str(value) for value in Data.Wide.unique()), 'projections stitched laterally)', end=' ')
print('at every',
      str(" or ".join(str(value) for value in Data.RotationStep.unique())) + '° over a 180° sample rotation.')

For each sample, we recorded a set of 2 or 3 stacked scans overlapping the sample height, each stack was recorded with 948 projections of 2452 x 1640 pixels at every 0.2° over a 180° sample rotation.


In [397]:
Data.Exposure.mean()

2286.0

In [396]:
print('Every single projection was exposed for',
      " or ".join(str(value) for value in Data.Exposure.unique()),
      'ms,',
      " or ".join(str(value) for value in Data.Averaging.unique()),
      'projections were averaged to one to greatly reduce image noise.')

Every single projection was exposed for 2349 or 1593 ms, 3 projections were averaged to one to greatly reduce image noise.


In [404]:
log=Data.LogFile[1]

In [405]:
print('This resulted in a scan time of approximately ', end='')
if duration(log)/3600 > 1:
    # Scan took hours
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
else:
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{minutes} minutes'), end=' ')
if not stacks(log) == 1:
    print('per stack and about',
          timeformat(stacks(log) * datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
print('per sample', end='')
if stacks(log) == 1:
    print('.')
else:
    print(' (with', stacks(log), 'stacks).')

This resulted in a scan time of approximately 2 hours and 48 minutes per stack and about 5 hours and 36 minutes per sample (with 2 stacks).


In [406]:
print('In total, we scanned', Data.Stacks.sum(), 'stacks.')
print('Each stack took approximately',
      Data.Duration.mean() // 60,
      'minutes (' + str(datetime.timedelta(seconds=Data.Duration.mean())) + ')')
print('In total, we thus scanned for about', 
      timeformat(Data.Stacks.sum() *
                 datetime.timedelta(seconds=Data.Duration.mean()),
                 '{days} days, {hours} hours and {minutes} minutes.'))
print('At the MIC rate, this would have cost',
      int(round(Data.Stacks.sum() * Data.Duration.mean() / 60 / 60 * 75)),
      'CHF!')

In total, we scanned 26 stacks.
Each stack took approximately 164.0 minutes (2:44:37.250000)
In total, we thus scanned for about 2 days, 23 hours and 20 minutes.
At the MIC rate, this would have cost 5350 CHF!


In [407]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      Data.Version.unique()[0][0],
      '(Version',
      version(log)[1] + ', Bruker microCT, Kontich Belgium)', end=' ')
if ringremoval(log):
    print('using a ring artifact correction of',
          ringremoval(log), end='')
if beamhardening(log):
    print(' and a beam hardening correction of',
          beamhardening(log),
          '%.')
else:
    print('.')
print('The whole process resulted in datasets with an isometric voxel size of',
      " or ".join(str(value) for value in Data.Voxelsize_rounded.unique()),
      'µm.')    

The projection images were then subsequently reconstructed into a 3D stack of images with NRecon (Version 1.7.4.6, Bruker microCT, Kontich Belgium) using a ring artifact correction of 14.
The whole process resulted in datasets with an isometric voxel size of 10.0 µm.


In [138]:
# fulllog(log)

In [282]:
Data.Voxelsize.mean()

10.000035999999998

In [283]:
Data.Beamhardening

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     8
10    0
11    0
Name: Beamhardening, dtype: int64